In [21]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import json


In [22]:
headers={'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36', 'Accept-Language': 'en-US,en;q=0.5'}
url='https://www.imdb.com/chart/top/?ref_=nv_mv_250'
page = requests.get(url,headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')


In [23]:
films = soup.select('.ipc-metadata-list-summary-item__c')

In [24]:
films_url=[]
for film in films:

    films_url.append('https://www.imdb.com'+film.find('a').get('href'))

In [33]:
film_collect=[]
count=0

for film_url in films_url:
         count+=1
         
         try:
            s = requests.Session()
            film_page = s.get(film_url,headers=headers,timeout =20)
            film_soup=BeautifulSoup(film_page.content, 'html.parser')



            result = {}

            result['Movie_id']=film_url.split("/")[4][2:]

            result['Title']=film_soup.find('h1').get_text()

            result['Year']=film_soup.find(class_='sc-69e49b85-0').select('a')[0].get_text()

            a=film_soup.find(class_='sc-69e49b85-0').select('a')
            if len(a)==1 :
                result['Parental_Guide']=None
            else: 
                result['Parental_Guide']=a[1].get_text()
 
            b=film_soup.find(class_='sc-69e49b85-0').find_all(class_='ipc-inline-list__item')
            if len(b)==2 :
                result['Runtime']=b[1].get_text()
            else: 
                result['Runtime']=b[2].get_text()

            genres=film_soup.find(class_='sc-69e49b85-4').find_all(class_='ipc-chip__text')
            result['Genre']=[genre.get_text() for genre in genres]

            directors=film_soup.find(class_='sc-69e49b85-2').find_all(class_='ipc-metadata-list-item__content-container')[0].find_all('a')
            result['Director']=[(director.get('href').split('/')[2][2:],director.get_text()) for director in directors]
 
            writers=film_soup.find(class_='sc-69e49b85-2').find_all(class_='ipc-metadata-list-item__content-container')[1].find_all('a')
            result['Writer']=[(writer.get('href').split('/')[2][2:],writer.get_text()) for writer in writers]

            stars=film_soup.find(class_='sc-69e49b85-2').find_all(class_='ipc-metadata-list-item__content-container')[2].find_all('a')
            result['Star']=[(star.get('href').split('/')[2][2:],star.get_text()) for star in stars]

            g=film_soup.find_all(class_='sc-f65f65be-0 bBlII')
            result['Gross_US_Canada']=None
            for i in range(len(g)):
                   c=g[i].find_all('span')
                   for j in range(len(c)):
                        if c[j].get_text()=='Gross US & Canada':
                              result['Gross_US_Canada']=c[j+1].get_text()

            film_collect.append(result)
         except Exception as e:
              print(count,e)
      

In [34]:
s=pd.DataFrame(film_collect)
s

,Movie_id,Title,Year,Parental_Guide,Runtime,Genre,Director,Writer,Star,Gross_US_Canada
0,0111161,The Shawshank Redemption,1994,R,2h 22m,[Drama],"[(0001104, Frank Darabont)]","[(0000175, Stephen King), (0001104, Frank Dara...","[(0000209, Tim Robbins), (0000151, Morgan Free...","$28,767,189"
1,0068646,The Godfather,1972,R,2h 55m,"[Crime, Drama]","[(0000338, Francis Ford Coppola)]","[(0701374, Mario Puzo), (0000338, Francis Ford...","[(0000008, Marlon Brando), (0000199, Al Pacino...","$136,381,073"
2,0468569,The Dark Knight,2008,PG-13,2h 32m,"[Action, Crime, Drama]","[(0634240, Christopher Nolan)]","[(0634300, Jonathan Nolan), (0634240, Christop...","[(0000288, Christian Bale), (0005132, Heath Le...","$534,987,076"
3,0071562,The Godfather Part II,1974,R,3h 22m,"[Crime, Drama]","[(0000338, Francis Ford Coppola)]","[(0000338, Francis Ford Coppola), (0701374, Ma...","[(0000199, Al Pacino), (0000134, Robert De Nir...","$47,834,595"
4,0050083,12 Angry Men,1957,Approved,1h 36m,"[Crime, Drama]","[(0001486, Sidney Lumet)]","[(0741627, Reginald Rose)]","[(0000020, Henry Fonda), (0002011, Lee J. Cobb...",None
...,...,...,...,...,...,...,...,...,...,...
245,1454029,The Help,2011,PG-13,2h 26m,[Drama],"[(0853238, Tate Taylor)]","[(0853238, Tate Taylor), (3543826, Kathryn Sto...","[(0205626, Viola Davis), (1297015, Emma Stone)...","$169,708,112"
246,0025316,It Happened One Night,1934,Passed,1h 45m,"[Comedy, Romance]","[(0001008, Frank Capra)]","[(0728307, Robert Riskin), (0011343, Samuel Ho...","[(0000022, Clark Gable), (0001055, Claudette C...",None
247,0053198,The 400 Blows,1959,Not Rated,1h 39m,"[Crime, Drama]","[(0000076, François Truffaut)]","[(0000076, François Truffaut), (0610115, Marce...","[(0529543, Jean-Pierre Léaud), (0753716, Alber...",$509
248,4430212,Drishyam,2015,Not Rated,2h 43m,"[Crime, Drama, Mystery]","[(1723225, Nishikant Kamat)]","[(2572228, Jeethu Joseph), (3179723, Upendra S...","[(0222426, Ajay Devgn), (1328152, Shriya Saran...",None


In [35]:
with open("imdb-scraped.json", "w") as file:
    json.dump(film_collect, file)